## Lab 08: Expenses and Budgeting

Welcome to the eigth lab of the course; we will do exercises on data structures and comprehension syntax simulating analysis of expenses.

## Exercise 1: Getting started

Use comprehension syntax on `list_expenses` to
- create a list where only the amount is kept
- create a list where only expenses made to Uber are kept
- create the set of dates in dd/mm/yyyy format (hint: split the date on '-' and concatenate the different pieces)
- create a list of tuples with two elements: the price (rounded to no decimals) and the length of the string describing the recipient (second element of the tuple); this, only if the tuple in `list_expenses` contains the word "util" in the category of the expense
- create the set of categories for which there is at least an expense of 60 euros

In [ ]:
list_expenses = [
    ('2024-10-01','Starbucks',5.75,'Food & Drink'),
    ('2024-10-05','Amazon',35.50,'Shopping'),
    ('2024-10-08','Local Restaurant',28.75,'Food & Drink'),
    ('2024-10-08','Uber',12.35,'Transportation'),
    ('2024-10-08','Electric Company',102.45,'Utilities'),
    ('2024-10-13','Walmart',68.20,'Groceries'),
    ('2024-10-13','Cinema',13.99,'Entertainment'),
    ('2024-10-13','Starbucks',4.50,'Food & Drink'),
    ('2024-10-22','Walmart',78.90,'Groceries'),
    ('2024-10-22','Water Utility',60.25,'Utilities'),
    ('2024-10-25','Amazon',25.99,'Shopping'),
    ('2024-10-25','Uber',10.75,'Transportation')
]

In [ ]:
[ t[2] for t in list_expenses ]
[ t for t in list_expenses if t[1]=='Uber']
{ t[0].split('-')[2]+"/"+t[0].split('-')[1]+t[0].split('-')[0] for t in list_expenses}
# alternative: { "/".join(reversed(item[0].split('-'))) for item in list_expenses }
# alternative: { "/".join(item[0].split('-')[::-1]) for item in list_expenses }
[ (round(t[2]), len(t[1])) for t in list_expenses if "util" in t[1].lower() ]
{ t[3] for t in list_expenses if t[2]>=60}

## Exercise 2: Merging expenses

If you have multiple bank accounts, each of themm will give you their own list of expenses, using different structures. The goal here is to merge them into a single, uniform list.

### 2.1 Mapping functions

Assume the existence of a mapping dictionary (see below) to be used to replace keys and values in the list of expenses. Each dictionary item is structured as 
    
    mapping['correct_key'] = (set_of_wrong_keys_to_be_replaced_with_correct_key, mapping_function_to_be_applied_to_values_with_wrong_keys)

Complete the mapping dictionary by:
- Creating a function that takes only the date part in a string that contains datetime field
    - E.g., `date_polish("2024-01-01 08:15")` must return '2024-01-01'
- Adding a lambda functiont to the 'amount' key that takes in input a number and returns its value rounded to 2 decimals

In [1]:
def date_polish(date):
    return date[:10]

In [ ]:
date_polish("2024-01-01 08:15")

In [2]:
mapping = dict()
mapping['date'] = ({'datetime'}, date_polish)
mapping['amount'] = ({'expense','value'}, lambda v: round(v,2))

### 2.2 Count the number of items that need to be resolved

- Create a set as the union of the values in the mapping dictionary
- Iterate on the lists to verify if an item contains one of the keys that should be replaced

In [3]:
lists = [[
    {'date': '2024-10-12', 'recipient': 'Starbucks', 'description': 'Coffee', 'amount': 495.37, 'category': 'Food & Drink'},
    {'date': '2024-10-15', 'recipient': 'Starbucks', 'description': 'Coffee', 'amount': 484.96, 'category': 'Food & Drink'},
    {'date': '2024-10-05', 'recipient': 'Starbucks', 'description': 'Coffee', 'value': 338.41003, 'category': 'Food & Drink'}
],[
    {'datetime': '2024-10-17 09:00', 'recipient': 'Uber', 'description': 'Fuel', 'amount': 200.44545, 'category': 'Transportation'},
    {'datetime': '2024-10-10 10:30', 'recipient': 'Apple Store', 'description': 'Gadget purchase', 'amount': 69.46769, 'category': 'Electronics'},
    {'datetime': '2024-10-21 11:45', 'recipient': 'Amazon', 'description': 'Online purchase', 'expense': 213.89211, 'category': 'Shopping'}
]]

In [8]:
combined_set = set()
for k in mapping.keys():
    combined_set = combined_set | mapping[k][0]
    
combined_set

{'datetime', 'expense', 'value'}

In [10]:
cnt = 0
for list in lists:
    for item in list:
        wrong_label_found = False
        for k in combined_set:
            if k in item.keys():
                wrong_label_found = True
                print(f'I found here that item {item} contains {k}')
        if wrong_label_found:
            # The counter is increased only once per item
            cnt += 1
cnt

I found here that item {'date': '2024-10-05', 'recipient': 'Starbucks', 'description': 'Coffee', 'value': 338.41003, 'category': 'Food & Drink'} contains value
I found here that item {'datetime': '2024-10-17 09:00', 'recipient': 'Uber', 'description': 'Fuel', 'amount': 200.44545, 'category': 'Transportation'} contains datetime
I found here that item {'datetime': '2024-10-10 10:30', 'recipient': 'Apple Store', 'description': 'Gadget purchase', 'amount': 69.46769, 'category': 'Electronics'} contains datetime
I found here that item {'datetime': '2024-10-21 11:45', 'recipient': 'Amazon', 'description': 'Online purchase', 'expense': 213.89211, 'category': 'Shopping'} contains expense
I found here that item {'datetime': '2024-10-21 11:45', 'recipient': 'Amazon', 'description': 'Online purchase', 'expense': 213.89211, 'category': 'Shopping'} contains datetime


4

In [12]:
# Alternative, more compact version
cnt = 0
for list in lists:
    for item in list:
        if len(item.keys() & combined_set) > 0:
            cnt += 1
            print(f'I found here that item {item} contains something that labels {item.keys() & combined_set} must be changed')
cnt

I found here that item {'date': '2024-10-05', 'recipient': 'Starbucks', 'description': 'Coffee', 'value': 338.41003, 'category': 'Food & Drink'} contains something that labels {'value'} must be changed
I found here that item {'datetime': '2024-10-17 09:00', 'recipient': 'Uber', 'description': 'Fuel', 'amount': 200.44545, 'category': 'Transportation'} contains something that labels {'datetime'} must be changed
I found here that item {'datetime': '2024-10-10 10:30', 'recipient': 'Apple Store', 'description': 'Gadget purchase', 'amount': 69.46769, 'category': 'Electronics'} contains something that labels {'datetime'} must be changed
I found here that item {'datetime': '2024-10-21 11:45', 'recipient': 'Amazon', 'description': 'Online purchase', 'expense': 213.89211, 'category': 'Shopping'} contains something that labels {'expense', 'datetime'} must be changed


4

### 2.3 Make consistent items

- Unify the lists: instead of a list of lists of items, make it a single list of items
- Iterate on the list to replace wrong keys with correct keys; use `item.pop(key)` to remove a key

In [13]:
new_list = []
for l in lists:
    new_list.extend(l)
    # alternative: new_list += l

In [16]:
# First solution: for every item, iterate on the dictionary to check whether the wrong keys exist in the item
for item in new_list:
    for correct_key in mapping.keys():
        for wrong_key in mapping[correct_key][0]:
            if wrong_key in item:
                item[correct_key] = mapping[correct_key][1](item[wrong_key])
                item.pop(wrong_key)

new_list

[{'date': '2024-10-12', 'recipient': 'Starbucks', 'description': 'Coffee', 'amount': 495.37, 'category': 'Food & Drink'}, {'date': '2024-10-15', 'recipient': 'Starbucks', 'description': 'Coffee', 'amount': 484.96, 'category': 'Food & Drink'}, {'date': '2024-10-05', 'recipient': 'Starbucks', 'description': 'Coffee', 'category': 'Food & Drink', 'amount': 338.41}, {'recipient': 'Uber', 'description': 'Fuel', 'amount': 200.44545, 'category': 'Transportation', 'date': '2024-10-17'}, {'recipient': 'Apple Store', 'description': 'Gadget purchase', 'amount': 69.46769, 'category': 'Electronics', 'date': '2024-10-10'}, {'recipient': 'Amazon', 'description': 'Online purchase', 'category': 'Shopping', 'date': '2024-10-21', 'amount': 213.89}]


In [18]:
# Second solution: for every item, iterate on its keys to check whether they are indicated in the mapping as wrong keys

# This solution may give errors on the third line
# In general, iterating over something (in this case, the mapping.keys()) that is being updated (in this case, by adding new keys) is not good practice
# Since sets do not have a fixed order, the error may or may not show up - you will see this by restarting the kernel multiple times
# for item in new_list:
#     for orig_key in item.keys():
#         for correct_key in mapping.keys():
#             if orig_key in mapping[correct_key][0]:
#                 item[correct_key] = mapping[correct_key][1](item[orig_key])
    
# To avoid the error above, new key-value pairs are first put in a temporary new_item and then added to the original item
for item in new_list:
    new_item = {}
    for orig_key in item.keys():
        for correct_key in mapping.keys():
            if orig_key in mapping[correct_key][0]:
                new_item[correct_key] = mapping[correct_key][1](item[orig_key])
    
    for new_key in new_item:
        item[new_key] = new_item[new_key]
        
new_list

[{'date': '2024-10-12',
  'recipient': 'Starbucks',
  'description': 'Coffee',
  'amount': 495.37,
  'category': 'Food & Drink'},
 {'date': '2024-10-15',
  'recipient': 'Starbucks',
  'description': 'Coffee',
  'amount': 484.96,
  'category': 'Food & Drink'},
 {'date': '2024-10-05',
  'recipient': 'Starbucks',
  'description': 'Coffee',
  'category': 'Food & Drink',
  'amount': 338.41},
 {'recipient': 'Uber',
  'description': 'Fuel',
  'amount': 200.44545,
  'category': 'Transportation',
  'date': '2024-10-17'},
 {'recipient': 'Apple Store',
  'description': 'Gadget purchase',
  'amount': 69.46769,
  'category': 'Electronics',
  'date': '2024-10-10'},
 {'recipient': 'Amazon',
  'description': 'Online purchase',
  'category': 'Shopping',
  'date': '2024-10-21',
  'amount': 213.89}]

## Exercise 3: Analyzing expenses

Now that you have a single, uniform list of expenses, we can do some analysis

In [8]:
list_expenses = [
    ('2024-10-01','Starbucks',5.75,'Food & Drink'),
    ('2024-10-01','Walmart',82.45,'Groceries'),
    ('2024-10-01','Netflix',15.99,'Entertainment'),
    ('2024-10-04','Shell Gas',45.60,'Transportation'),
    ('2024-10-05','Apple Store',49.99,'Electronics'),
    ('2024-10-05','Amazon',35.50,'Shopping'),
    ('2024-10-08','Local Restaurant',28.75,'Food & Drink'),
    ('2024-10-08','Uber',12.35,'Transportation'),
    ('2024-10-08','Electric Company',102.45,'Utilities'),
    ('2024-10-13','Walmart',68.20,'Groceries'),
    ('2024-10-13','Cinema',13.99,'Entertainment'),
    ('2024-10-13','Starbucks',4.50,'Food & Drink'),
    ('2024-10-13','Walmart',90.67,'Groceries'),
    ('2024-10-14','Local Restaurant',22.15,'Food & Drink'),
    ('2024-10-15','Netflix',15.99,'Entertainment'),
    ('2024-10-16','Target',58.90,'Shopping'),
    ('2024-10-17','Shell Gas',52.10,'Transportation'),
    ('2024-10-22','Apple Store',19.99,'Electronics'),
    ('2024-10-22','Local Gym',40.00,'Health'),
    ('2024-10-22','Starbucks',6.50,'Food & Drink'),
    ('2024-10-22','Walmart',78.90,'Groceries'),
    ('2024-10-22','Water Utility',60.25,'Utilities'),
    ('2024-10-25','Amazon',25.99,'Shopping'),
    ('2024-10-25','Uber',10.75,'Transportation'),
    ('2024-10-25','Cinema',15.50,'Entertainment'),
    ('2024-10-26','Local Restaurant',30.10,'Food & Drink'),
    ('2024-10-27','Walmart',85.30,'Groceries'),
    ('2024-10-28','Shell Gas',47.25,'Transportation'),
    ('2024-10-30','Electric Company',101.85,'Utilities'),
    ('2024-10-30','Starbucks',5.25,'Food & Drink')
]

category_acronyms = {
    'Food & Drink': 'F&D',
    'Groceries': 'GRO',
    'Entertainment': 'ENT',
    'Transportation': 'TRN',
    'Electronics': 'ELE',
    'Shopping': 'SHP',
    'Utilities': 'UTL',
    'Health': 'HLT'
}

### 3.1 Comprehension

Use comprehension syntax to create a list of tuples where each tuples is composed by:
- The day of the date
- The description of the expense in reverse order,
- The biggest multiple of ten that is lower than the expense (i.e., remove from the expense value the result of the "modulo 10" operation, then round to 0 decimals)
- The acronym of the category

In [2]:
[ (t[0].split('-')[2], t[1][::-1], round(t[2]-t[2]%10), category_acronyms[t[3]]) for t in list_expenses ]

[('01', 'skcubratS', 0, 'F&D'),
 ('01', 'tramlaW', 80, 'GRO'),
 ('01', 'xilfteN', 10, 'ENT'),
 ('04', 'saG llehS', 40, 'TRN'),
 ('05', 'erotS elppA', 40, 'ELE'),
 ('05', 'nozamA', 30, 'SHP'),
 ('08', 'tnaruatseR lacoL', 20, 'F&D'),
 ('08', 'rebU', 10, 'TRN'),
 ('08', 'ynapmoC cirtcelE', 100, 'UTL'),
 ('13', 'tramlaW', 60, 'GRO'),
 ('13', 'ameniC', 10, 'ENT'),
 ('13', 'skcubratS', 0, 'F&D'),
 ('13', 'tramlaW', 90, 'GRO'),
 ('14', 'tnaruatseR lacoL', 20, 'F&D'),
 ('15', 'xilfteN', 10, 'ENT'),
 ('16', 'tegraT', 50, 'SHP'),
 ('17', 'saG llehS', 50, 'TRN'),
 ('22', 'erotS elppA', 10, 'ELE'),
 ('22', 'myG lacoL', 40, 'HLT'),
 ('22', 'skcubratS', 0, 'F&D'),
 ('22', 'tramlaW', 70, 'GRO'),
 ('22', 'ytilitU retaW', 60, 'UTL'),
 ('25', 'nozamA', 20, 'SHP'),
 ('25', 'rebU', 10, 'TRN'),
 ('25', 'ameniC', 10, 'ENT'),
 ('26', 'tnaruatseR lacoL', 30, 'F&D'),
 ('27', 'tramlaW', 80, 'GRO'),
 ('28', 'saG llehS', 40, 'TRN'),
 ('30', 'ynapmoC cirtcelE', 100, 'UTL'),
 ('30', 'skcubratS', 0, 'F&D')]

### 3.2 Expenses by category

Create a dictionary `expenses_by_category` where:
- Keys are the categories
- Values correspond to the total expenses of that category

In [3]:
expenses_by_category = dict()

for ex in list_expenses:
    cat = ex[3]
    if cat in expenses_by_category:
        expenses_by_category[cat] = expenses_by_category[cat] + ex[2]
    else:
        expenses_by_category[cat] = ex[2]

expenses_by_category

{'Food & Drink': 103.0,
 'Groceries': 405.52000000000004,
 'Entertainment': 61.47,
 'Transportation': 168.05,
 'Electronics': 69.98,
 'Shopping': 120.39,
 'Utilities': 264.54999999999995,
 'Health': 40.0}

### 3.3 Percentage of expenses by category

Create a dictionary where 
- Keys are the categories
- Values are the ratio of total expenses of that category over the total expenses in the list

Hint: compute the totals by category as in 3.2, then use dictionary comprehension to compute percentages

In [9]:
expenses_by_category = dict()
total = 0

for ex in list_expenses:
    cat = ex[3]
    if cat in expenses_by_category:
        expenses_by_category[cat] = expenses_by_category[cat] + ex[2]
    else:
        expenses_by_category[cat] = ex[2]
    total += ex[2]

{ k:str(round(v*100/total,2))+"%" for k,v in expenses_by_category.items()}

{'Food & Drink': '8.35%',
 'Groceries': '32.89%',
 'Entertainment': '4.99%',
 'Transportation': '13.63%',
 'Electronics': '5.68%',
 'Shopping': '9.76%',
 'Utilities': '21.46%',
 'Health': '3.24%'}

### 3.4 Budgeting

Assume you have a budget for each category, specified in a dictionary. Now, we want to create a new budget for the next month. Here are two different strategy.
- Create a new budget, 'budget_v1', where the maximum expenditure for each category is adjusted based on the expenses of the previous month.
    - If you had a budget of 100 and spent 110, the budget for next month will be 90
    - If you had a budget of 100 and spent 90, the budget for next month will be 110
- Create a new budget, 'budget_v1', where you adjust the maximum expenditure of a category only if you had overspent by 10 euroes or more.
    - If you had a budget of 100 and spent 110, the budget for next month will be 90
    - If you had a budget of 100 and spent 109 or less, the budget for next month will still be 100

Hint: in both cases, compute the totals by category as in 3.2, then rely only on comprehension syntax

In [5]:
budget = {
    'Food & Drink': 100,
    'Groceries': 500,
    'Entertainment': 100,
    'Transportation': 150,
    'Electronics': 70,
    'Shopping': 100,
    'Utilities': 250,
    'Health': 100
}

In [6]:
expenses_by_category = dict()

for ex in list_expenses:
    cat = ex[3]
    if cat in expenses_by_category:
        expenses_by_category[cat] = expenses_by_category[cat] + ex[2]
    else:
        expenses_by_category[cat] = ex[2]

budget_v1 = { cat:bud+round(bud-expenses_by_category[cat],2) for cat,bud in budget.items()}
budget_v1

{'Food & Drink': 97.0,
 'Groceries': 594.48,
 'Entertainment': 138.53,
 'Transportation': 131.95,
 'Electronics': 70.02,
 'Shopping': 79.61,
 'Utilities': 235.45,
 'Health': 160.0}

In [7]:
expenses_by_category = dict()

for ex in list_expenses:
    cat = ex[3]
    if cat in expenses_by_category:
        expenses_by_category[cat] = expenses_by_category[cat] + ex[2]
    else:
        expenses_by_category[cat] = ex[2]

d = { cat:bud+round(bud-expenses_by_category[cat],2) for cat,bud in budget.items() if round(bud-expenses_by_category[cat],2)<=-10}
{ cat: d.get(cat,bud) for cat,bud in budget.items() }

{'Food & Drink': 100,
 'Groceries': 500,
 'Entertainment': 100,
 'Transportation': 131.95,
 'Electronics': 70,
 'Shopping': 79.61,
 'Utilities': 235.45,
 'Health': 100}